In [ ]:
lvl           = None
file_path_Lvl = None

In [ ]:
%run ./6___Levels_isolated_pairs___Functions.ipynb

---
---
---

### Find all the isolated cells

We do this by checking if a cell in the leveled cube is equal to all of its neighbors by means of subtracting the offset cube 26 times.

In [ ]:
with open(file_path_Lvl+"grid_lvld.pk", 'rb') as f: grid_lvld = pkl.load(f)

In [ ]:
# Grid of all isolated cells (including origins)
grid_lvld_isolated = np.full(grid_lvld.shape, True)

for offs in offsets: grid_lvld_isolated &= (grid_lvld != shift_cube(grid_lvld, offs))

### Dict of lists for all levels of the coordinates of isolated cells in them

In [ ]:
# Make all not-isolated cells = -1.
grid_lvld[~grid_lvld_isolated] = -1

In [ ]:
# We found this to be a good limit given our RAM.
max_lvls_per_list = round(lvl / 100 * (512/grid_lvld.shape[0])**3)
no_lists = lvl // max_lvls_per_list + (1 if (lvl % max_lvls_per_list)!=0 else 0)

In [ ]:
for list_index in range(no_lists):

    dict_cells_isolated = {val: [] for val in range(lvl)}
    
    for coord in np.argwhere((max_lvls_per_list * list_index         <= grid_lvld) & 
                             (max_lvls_per_list * (list_index+1) - 1 >= grid_lvld)):
        
        dict_cells_isolated[grid_lvld[tuple(coord)]].append(coord.tolist())
    
    with open(file_path_Lvl+"dict_cells_isolated/"+str(list_index)+".pk", 'wb') as f: pkl.dump(dict_cells_isolated, f)

In [ ]:
del dict_cells_isolated; gc.collect()

---

### Get the pairs cells

In [ ]:
with open(file_path_Lvl+"grid_lvld.pk", 'rb') as f: grid_lvld = pkl.load(f)

# Make all isolated cells = -1.
grid_lvld[grid_lvld_isolated] = -1

In [ ]:
for list_index in range(no_lists):
    
    dict_cells_pairs = {val: [] for val in range(lvl)}

    for coord in np.argwhere((max_lvls_per_list * list_index         <= grid_lvld) & 
                             (max_lvls_per_list * (list_index+1) - 1 >= grid_lvld)):
        
        dict_cells_pairs[grid_lvld[tuple(coord)]].append(coord.tolist())
    
    # Now make pairs within each level's pairs cells
    for i in range(max_lvls_per_list * list_index, np.min([max_lvls_per_list * (list_index+1), lvl])):
        ldcp = len(dict_cells_pairs[i])
        if ldcp != 0:
            if 2 <= ldcp <= 3:
                # We know all cells are part of a pair, so if there are two, they are the pair and if there are
                #     3, you can't just leave one outside of the pair, it'd be isolated, so they are all part of a
                #     single pair.
                dict_cells_pairs[i] = [dict_cells_pairs[i]]
            else:
                dict_cells_pairs[i] = pairs_finder_1(np.array(dict_cells_pairs[i]), grid_lvld.shape[0])

    with open(file_path_Lvl+"dict_cells_pairs/"+str(list_index)+".pk", 'wb') as f: pkl.dump(dict_cells_pairs, f)

---
---
---